Consider the diabetes.arff data available at the homework tab, comprising 8 biological features to 
classify 768 patients into 2 classes (normal, diabetes).  
 
 
1) [1v] ANOVA is a statistical test that can be used to assess the discriminative power of a 
single input variable. Using f_Outcomeif from sklearn, identify the input variables with the 
worst and best discriminative power. Plot their Outcome-conditional probability density 
functions.

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import arff
from sklearn.feature_selection import f_classif
import seaborn as sns

# Load the diabetes dataset from the ARFF file
data, meta = arff.loadarff('diabetes.arff')
df = pd.DataFrame(data)

# Convert byte strings to regular strings in the class column
df['Outcome'] = df['Outcome'].apply(lambda x: x.decode('utf-8'))

# Separate the features and target variable
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Perform ANOVA to assess discriminative power of each featf_classiflues = f_classif(X, y)[0]

best_feature = df.columns[F_values.argmax()]
worst_feature = df.columns[F_values.argmin()]

print(f"Best Feature: {best_feature}")
print(f"Worst Feature: {worst_feature}")

# Plot class-conditional probability density functions for best and worst features
plt.figure(figsize=(10, 5))

# Plot for the best feature
plt.subplot(1, 2, 1)
sns.kdeplot(data=df, x=best_feature, hue='Outcome', common_norm=False, fill=True)
plt.title(f'Outcome-Conditional PDF of Best Feature: {best_feature}')
plt.xlabel(best_feature)
plt.ylabel('Density')

# Plot for the worst feature
plt.subplot(1, 2, 2)
sns.kdeplot(data=df, x=worst_feature, hue='Outcome', common_norm=False, fill=True)
plt.title(f'Outcome-Conditional PDF of Worst Feature: {worst_feature}')
plt.xlabel(worst_feature)
plt.ylabel('Deaaaaaaa

plt.tight_layout()
plt.show()
Outcome

In [28]:
import pandas as pd
from scipy.io import arff
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np

# Load the diabetes dataset from the ARFF file
# Replace 'path_to_diabetes.arff' with the correct path to the file on your system
data, meta = arff.loadarff('diabetes.arff')
df = pd.DataFrame(data)

# Convert byte strings to regular strings in the class column
df['Outcome'] = df['Outcome'].apply(lambda x: x.decode('utf-8'))

# Separate the features (X) and target variable (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Set the minimum sample split values to be tested
min_samples_splits = [2, 5, 10, 20, 30, 50, 100]

# Initialize lists to store accuracies
train_accuracies = []
test_accuracies = []

# Perform 10 runs to average out the randomness in the split thresholding (optional)
n_runs = 10
for min_samples_split in min_samples_splits:
    train_acc_run = []
    test_acc_run = []
    
    for _ in range(n_runs):
        # Split the dataset into stratified 80-20 training-testing sets with a fixed random state
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)
        
        # Create and train the Decision Tree classifier
        clf = DecisionTreeClassifier(min_samples_split=min_samples_split, random_state=1)
        clf.fit(X_train, y_train)
        
        # Calculate training accuracy
        y_train_pred = clf.predict(X_train)
        train_acc = accuracy_score(y_train, y_train_pred)
        train_acc_run.append(train_acc)
        
        # Calculate testing accuracy
        y_test_pred = clf.predict(X_test)
        test_acc = accuracy_score(y_test, y_test_pred)
        test_acc_run.append(test_acc)
    
    # Average the accuracies over the runs
    train_accuracies.append(np.mean(train_acc_run))
    test_accuracies.append(np.mean(test_acc_run))

# Plotting the training and testing accuracies
plt.figure(figsize=(10, 6))
plt.plot(min_samples_splits, train_accuracies, marker='o', linestyle='-', color='blue', label='Training Accuracy')
plt.plot(min_samples_splits, test_accuracies, marker='o', linestyle='--', color='red', label='Testing Accuracy')
plt.title('Training and Testing Accuracies of Decision Tree')
plt.xlabel('Minimum Samples Split')
plt.ylabel('Accuracy')
plt.xticks(min_samples_splits)
plt.grid(True)
plt.legend()
plt.show()
